C:\Users\christo.strydom\github_repos\automation\CBR\Copy of 20210104_Master_Gap_CBR_LE_CS.xlsx

In [1]:
import glob

In [2]:
import pyperclip
pyperclip.copy('The text to be copied to the clipboard.')
spam = pyperclip.paste()

In [3]:
spam

'The text to be copied to the clipboard.'

In [4]:
import os, os.path
from os import listdir
from os.path import isfile, join

In [5]:
import pandas as pd
import openpyxl
import win32com.client
from openpyxl import load_workbook

In [6]:
from datetime import datetime, timedelta
from dateutil.relativedelta import *

In [7]:
import calendar
import dateutil.parser

In [8]:
calendar.month_name[3]

'March'

In [9]:
path="""c:\\Users\\christo.strydom\\github_repos\\automation\\CBR\\data\\"""
active_policies_file='active_policies_20210304.csv'
new_policies_file='new_policies_20210304.csv'
gwp_new_policies_file='gwp_new_policies_20210304.csv'
gross_premium_file='gross_premium_20210307.csv'
cancellations_file='cancellations_20210304.csv'


In [10]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)

def last_day_of_month_fn(any_day):
    # this will never fail
    # get close to the end of the month for any day, and add 4 days 'over'
    next_month = any_day.replace(day=28) + timedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - timedelta(days=next_month.day)

def slice_saver(identifier_name, dataframe, save_path, datafile_name, product_name):
    s=datafile_name.split("_")[1].split(".")[0]
    product_name=product_name.lower().replace("-","").replace("  "," ").replace(" ","_")
    fname="cbr"+'_'+product_name+"_"+s+ "_"+identifier_name+".csv"
    dataframe.to_csv(path_or_buf=save_path+fname, sep=',', index=False)
    return fname

In [11]:
nmonths=5
today = datetime.today()
months_ago = today + relativedelta(months=-nmonths)
som_months_ago = datetime(months_ago.year,months_ago.month,1)
iterate_months=[som_months_ago+relativedelta(months=n) for n in range(nmonths+1)]
iterate_months

[datetime.datetime(2020, 10, 1, 0, 0),
 datetime.datetime(2020, 11, 1, 0, 0),
 datetime.datetime(2020, 12, 1, 0, 0),
 datetime.datetime(2021, 1, 1, 0, 0),
 datetime.datetime(2021, 2, 1, 0, 0),
 datetime.datetime(2021, 3, 1, 0, 0)]

Create mysql scripts for OWLS data extraction:

### Active policies
#### active_policies_20210304

In [12]:
def active_policies_fn(period):
    year=period.year
    month=str(period.month).zfill(2)
    ldm=calendar.monthrange(year, period.month)[1]
    today = datetime.today()
    d=str(today.day).zfill(2)
    m=str(today.month).zfill(2)
#     m = calendar.month_name[today.month]
    y=today.year   
    return """SELECT  '{year}/{month}/01' AS `period`
    ,'active_policies_{y}{m}{d}' AS `operation`  
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,productsetup.productsetupname AS `product`
    ,IF (policy.policygroupholdingsid ='' OR 
        (SELECT policygroupholdings.policygroupholdingsname            
        FROM policygroupholdings            
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%indiv%' 
        OR 
        (SELECT policygroupholdings.policygroupholdingsname 
        FROM policygroupholdings 
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%Private%','individual','corporate') as `grouptype`
    ,count(distinct policynumber) AS active_policy_count
    from policy,productsetup
    where (policy.cancellationdate is null OR DATE(policy.cancellationdate)>'{year}/{month}/{ldm}') and 
    policy.policytransactionid = 
    (
        SELECT p.policytransactionid 
        FROM policy p WHERE p.policynumber=policy.policynumber AND DATE(p.inceptiondate)<='{year}/{month}/{ldm}' 
        ORDER by p.updateddate DESC
        LIMIT 1) 
    group by `period`,`umaproductsetupid`,`grouptype`
    UNION
    select  '{year}/{month}/01' as `period`
    ,'active_policies_{y}{m}{d}' AS `operation`     
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,productsetup.productsetupname AS `product`
    ,'aggregate' as `grouptype`
    ,count(distinct policynumber) AS active_policy_count
    from policy,productsetup
    where (policy.cancellationdate is null OR DATE(policy.cancellationdate)>'{year}/{month}/{ldm}') and 
    policy.policytransactionid = 
    (
        SELECT p.policytransactionid 
        FROM policy p WHERE p.policynumber=policy.policynumber AND DATE(p.inceptiondate)<='{year}/{month}/{ldm}' 
        ORDER by p.updateddate DESC
        LIMIT 1) 
    group by `period`,`umaproductsetupid`,`grouptype`
    """.format(year=year,month=month,ldm=ldm,y=y,m=m,d=d)

In [13]:
def active_policies_query_fn(iterate_months):
    union_str=' UNION '
    sqlquery=''
    for iterate_month in iterate_months:
#         print(iterate_month)
        new_policies_query=active_policies_fn(iterate_month)
        if iterate_month!=iterate_months[-1]:
            sqlquery=sqlquery+new_policies_query+union_str
        else:
            sqlquery=sqlquery+new_policies_query
    return sqlquery

In [14]:
spam=pyperclip.copy(active_policies_query_fn(iterate_months))

In [15]:
active_policies_df=pd.read_csv(filepath_or_buffer=path+active_policies_file)
gross_premium_df=pd.read_csv(filepath_or_buffer=path+gross_premium_file)
new_policies_df=pd.read_csv(filepath_or_buffer=path+new_policies_file)
gwp_new_policies_df=pd.read_csv(filepath_or_buffer=path+gwp_new_policies_file)
cancellations_df=pd.read_csv(filepath_or_buffer=path+cancellations_file)
# gwp_new_policies_file='gwp_new_policies_20210304.csv'
# gross_premium_file='gross_premium_20210304.csv'
# cancellations_file='cancellations_20210304.csv'

In [16]:
product_list=list(set(active_policies_df['product']))

In [17]:
product_list 

['Sanlam Gap',
 'MedExpense',
 'Western Gap',
 'Western Gap Care',
 'Kaelo Gap',
 'Centriq Cancer']

In [18]:
file_name='cbr_template.xlsx'
file_path='C:\\Users\\christo.strydom\\github_repos\\automation\\CBR\\reports\\'
fn=file_path+file_name
writer = pd.ExcelWriter(fn, engine='openpyxl')
book = load_workbook(fn)
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

In [19]:
# presentation_path='\\\klo-jhb-fp01\\Xelus\\Data Science\\ChristoStrydom\\Presentations\\'
# presentation_name='cbr_presentation_template.xlsx'
# # file_name='cbr_presentation_template.xlsx'
# cbr_presentation_template = openpyxl.load_workbook(presentation_path+presentation_name) 
# summary_sheet = cbr_presentation_template["Sanlam Gap - Individual"]

In [20]:
active_policies_df['period']=pd.to_datetime(active_policies_df['period'], format='%Y/%m/%d', errors='ignore')
active_policies_df['year']=pd.DatetimeIndex(active_policies_df['period']).year
active_policies_df['month']=pd.DatetimeIndex(active_policies_df['period']).month
active_policies_df['mperiod']=active_policies_df.apply(lambda x: str(x['month'])+' '+str(x['year']), axis=1)

In [21]:
gross_premium_df['period']

0       2020/02/01
1       2020/02/01
2       2020/02/01
3       2020/02/01
4       2020/02/01
           ...    
4106    2021/03/07
4107    2021/03/07
4108    2021/03/07
4109    2021/03/07
4110    2021/03/31
Name: period, Length: 4111, dtype: object

In [22]:
gf=active_policies_df[(active_policies_df['product']=='Sanlam Gap')&(active_policies_df['grouptype']=='aggregate')].copy()


In [23]:
index=list()
data=list()
column_names=list(gf['mperiod'])
index.append('Active Policies')
data.append(list(gf['policy_count']))
df = pd.DataFrame(data, index=index, columns = column_names) 

In [24]:
index

['Active Policies']

In [25]:
gf=gross_premium_df[(gross_premium_df['product']=='Sanlam Gap')&(gross_premium_df['grouptype']=='aggregate')].copy()

In [26]:
df

,2 2020,3 2020,4 2020,5 2020,6 2020,7 2020,8 2020,9 2020,10 2020,11 2020,12 2020,1 2021,2 2021,3 2021
Active Policies,20452,20743,20791,21095,21252,21421,21572,21823,21867,22029,22259,22637,23354,23880


In [27]:
file_path='N:\\Data Science\\ChristoStrydom\\CBR reports\\'
dname=': Membership&Premium'
cname=': Claims'
writer = pd.ExcelWriter(fn, engine='openpyxl')
book = load_workbook(fn)
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

In [28]:
for f in onlyfiles:
    sheet_name=f.split('_')[1]
    print(f.split('_')[1])
    df=pd.read_csv(filepath_or_buffer=file_path+f)
    gf=df.loc[0:14].copy()
    gf.rename(columns={'Unnamed: 0': f.split('_')[1]+dname},inplace=True)
    hf=df.loc[15:27].copy()
    hf.rename(columns={'Unnamed: 0': f.split('_')[1]+cname},inplace=True)    
    gf.to_excel(writer, sheet_name=sheet_name, index=False, startcol=0,startrow=1)
    hf.to_excel(writer, sheet_name=sheet_name, index=False, startcol=0,startrow=19)
    writer.save()

NameError: name 'onlyfiles' is not defined

In [ ]:
file_name='cbr_template.xlsx'
file_path='C:\\Users\\christo.strydom\\github_repos\\automation\\CBR\\reports\\'
fn=file_path+file_name
writer = pd.ExcelWriter(fn, engine='openpyxl')
book = load_workbook(fn)
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

In [ ]:
writer.sheets

In [ ]:
product=product_list[0]

In [ ]:
print(product)
df.to_excel(writer, sheet_name=product, index=False, startcol=1,startrow=1)

In [ ]:
for product in product_list:
    print(product)
    df.to_excel(writer, sheet_name=sheet_name, index=False, startcol=0,startrow=0)

### Gross Premium
#### gross_premium_20210307

In [29]:
def gross_premium_raised_fn(iterate_months):
    start_year=iterate_months[0].year
    start_month=str(iterate_months[0].month).zfill(2)
    end_year=iterate_months[-1].year
    end_month=str(iterate_months[-1].month).zfill(2)    
    ldm=calendar.monthrange(end_year, iterate_months[-1].month)[1]
    today = datetime.today()
    d=str(today.day).zfill(2)
    m=str(today.month).zfill(2)
    y=today.year    
    return """
    SELECT left(fingl.posteddate,7) as `period` 
    ,policy.umaid AS `umaid`
    ,'gross_premium_{y}{m}{d}' AS `operation`      
    ,policy.umaproductsetupid AS `umaproductsetupid`
        ,productsetup.productsetupname AS `product`
    ,SUM(fingl.amount) AS `raised_premium_amount`
    ,COUNT(DISTINCT fingl.policynumber) AS `raised_premium_policycount`
    ,SUM(fingl.amount) / COUNT(DISTINCT fingl.policynumber) AS `raised_premium_average` 
    ,IF (policy.policygroupholdingsid ='' OR 
        (SELECT policygroupholdings.policygroupholdingsname            
        FROM policygroupholdings            
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%indiv%' 
        OR 
        (SELECT policygroupholdings.policygroupholdingsname 
        FROM policygroupholdings 
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%Private%','individual','corporate') AS `grouptype`
    from fingl,policy,productsetup
    where ( policy.policynumber = fingl.policynumber AND  
    fingl.currenthistoryflag = 'CURRENT' AND  
    policy.currenthistoryflag = 'CURRENT')  AND 
    (fingl.type='Premium Raised' and 
    fingl.posteddate BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{ldm}') 
    group by `period`,`umaproductsetupid`,`grouptype`
UNION
    SELECT left(fingl.posteddate,7) as `period`   
    ,policy.umaid AS `umaid`
    ,'gross_premium_{y}{m}{d}' AS `operation`      
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,productsetup.productsetupname AS `product`
    ,SUM(fingl.amount) AS `raised_premium_amount`
    ,COUNT(DISTINCT fingl.policynumber) AS `raised_premium_policycount`
    ,SUM(fingl.amount) / COUNT(DISTINCT fingl.policynumber) AS `raised_premium_average` 
    ,'aggregate' as `grouptype`
    from fingl,policy,productsetup
      where ( policy.policynumber = fingl.policynumber AND  
      fingl.currenthistoryflag = 'CURRENT' AND  
      policy.currenthistoryflag = 'CURRENT')  AND 
      (fingl.type='Premium Raised' and 
      fingl.posteddate BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{ldm}') 
      group by `period`,`umaproductsetupid`,`grouptype`
""".format(start_year=start_year,start_month=start_month,end_year=end_year,end_month=end_month,ldm=ldm,y=y,m=m,d=d)
    

In [30]:
# print(gross_premium_raised_fn(iterate_months))
spam=pyperclip.copy(gross_premium_raised_fn(iterate_months))

### New policies
#### new_policies_20210304

In [ ]:
'new_policies_'+str(today.year)+str(today.month).zfill(2)+str(today.day).zfill(2)

In [ ]:
def new_policies_fn(period):
    year=period.year
    month=str(period.month).zfill(2)
    ldm=calendar.monthrange(year, period.month)[1]
    today = datetime.today()
    d=str(today.day).zfill(2)
#     m = calendar.month_name[today.month]
    m=str(today.month).zfill(2)
    y=today.year   
    return """
    SELECT  '{year}/{month}/01' AS `period`
    ,'new_policies_{y}{m}{d}' AS `operation`   
    ,policy.umaid AS `umaid`
    ,productsetup.productsetupname AS `product`
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,IF (policy.policygroupholdingsid ='' OR 
        (SELECT policygroupholdings.policygroupholdingsname            
        FROM policygroupholdings            
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%indiv%' 
        OR 
        (SELECT policygroupholdings.policygroupholdingsname 
        FROM policygroupholdings 
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%Private%','Individual','Corporate') AS `grouptype`
    ,sum(policy.grosspremium) as `new_policy_gross_premium`
    ,count(distinct policy.policynumber) AS `new_policy_count`
    ,sum(policy.grosspremium) / count(distinct policy.policynumber) as `new_policy_average_gross_premium`
    from policy, productsetup
    where (policy.cancellationdate is null OR DATE(policy.cancellationdate)>'{year}/{month}/{ldm}') AND 
        (DATE(policy.inceptiondate)>='{year}/{month}/01' AND DATE(policy.inceptiondate)<='{year}/{month}/{ldm}') AND 
    policy.policytransactionid = 
    (
        SELECT p.policytransactionid 
        FROM policy p WHERE p.policynumber=policy.policynumber AND 
        (DATE(p.inceptiondate)>='{year}/{month}/01' AND DATE(p.inceptiondate)<='{year}/{month}/{ldm}') 
        ORDER by p.updateddate DESC
        LIMIT 1) 
    group by `umaproductsetupid`,`grouptype`
UNION
select  '{year}/{month}/01' AS `period`
,'new_policies_{y}{m}{d}' AS `operation`   
,policy.umaid AS `umaid`
    ,productsetup.productsetupname AS `product`
,policy.umaproductsetupid AS `umaproductsetupid`
,'aggregate' as `grouptype`
,sum(policy.grosspremium) as `new_policy_gross_premium`
,count(distinct policy.policynumber) AS `new_policy_count`
,sum(policy.grosspremium) / count(distinct policy.policynumber) as `new_policy_average_gross_premium`
from policy, productsetup
where (policy.cancellationdate is null OR DATE(policy.cancellationdate)>'{year}/{month}/{ldm}') AND 
    (DATE(policy.inceptiondate)>='{year}/{month}/01' AND DATE(policy.inceptiondate)<='{year}/{month}/{ldm}') AND 
policy.policytransactionid = 
(
    SELECT p.policytransactionid 
    FROM policy p WHERE p.policynumber=policy.policynumber AND 
    (DATE(p.inceptiondate)>='{year}/{month}/01' AND DATE(p.inceptiondate)<='{year}/{month}/{ldm}') 
    ORDER by p.updateddate DESC
    LIMIT 1) 
group by `umaproductsetupid`,`grouptype` 
     
    """.format(year=year,month=str(month).zfill(2),ldm=ldm,y=y,m=m,d=d)

In [ ]:
def new_policies_query_fn(iterate_months):
    union_str=' UNION '
    sqlquery=''
    for iterate_month in iterate_months:
#         print(iterate_month)
        new_policies_query=new_policies_fn(iterate_month)
        if iterate_month!=iterate_months[-1]:
            sqlquery=sqlquery+new_policies_query+union_str
        else:
            sqlquery=sqlquery+new_policies_query
    return sqlquery

In [ ]:
# print((iterate_months))
spam=pyperclip.copy(new_policies_query_fn(iterate_months))

### GWP of new policies
#### gwp_new_policies_20210303

In [ ]:
def gwp_new_policies_fn(period):
    year=period.year
    month=str(period.month).zfill(2)
    ldm=calendar.monthrange(year, period.month)[1]
    today = datetime.today()
    d=str(today.day).zfill(2)
#     m = calendar.month_name[today.month]
    m=str(today.month).zfill(2)
    y=today.year   
    return """
    SELECT  '{year}/{m}/01' AS `period_py`
    ,left(fingl.posteddate,7) AS `period`    
    ,'gwp_new_policies_{y}{m}{d}' AS `operation`    
        ,policy.umaid AS `umaid`
        ,productsetup.productsetupname AS `product`
        ,policy.umaproductsetupid AS `umaproductsetupid`
        ,IF (policy.policygroupholdingsid ='' OR 
            (SELECT policygroupholdings.policygroupholdingsname            
            FROM policygroupholdings            
            WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
            policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
            ) LIKE '%indiv%' 
            OR 
            (SELECT policygroupholdings.policygroupholdingsname 
            FROM policygroupholdings 
            WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
            policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
            ) LIKE '%Private%','Individual','Corporate') as `grouptype`

        ,sum(fingl.amount) as `gwp_new_policy_gross_premium`
        ,count(distinct policy.policynumber) AS `new_policy_count`
        ,sum(fingl.amount) / count(distinct policy.policynumber) as `gwp_new_policy_average_gross_premium`
        FROM fingl,policy,productsetup
        WHERE ( policy.policynumber = fingl.policynumber AND  
        fingl.currenthistoryflag = 'CURRENT' AND  
        policy.currenthistoryflag = 'CURRENT')  AND 
        (policy.cancellationdate is null OR DATE(policy.cancellationdate)>'{year}/{month}/{ldm}') AND 
        (DATE(policy.inceptiondate)>='{year}/{month}/01' AND DATE(policy.inceptiondate)<='{year}/{month}/{ldm}') AND
        (fingl.type='Premium Raised' AND 
        fingl.posteddate BETWEEN '{year}/{month}/01' AND '{year}/{month}/{ldm}') 
        group by `umaproductsetupid`,`grouptype`, `period`
    UNION
    SELECT  '{year}/{m}/01' AS `period_py`
        ,left(fingl.posteddate,7) AS `period`    
    ,'gwp_new_policies_{y}{m}{d}' AS `operation`    
        ,policy.umaid AS `umaid`
        ,productsetup.productsetupname AS `product`
        ,policy.umaproductsetupid AS `umaproductsetupid`
        ,'aggregate' as `grouptype`
        ,sum(fingl.amount) as `gwp_new_policy_gross_premium`
        ,count(distinct policy.policynumber) AS `new_policy_count`
        ,sum(fingl.amount) / count(distinct policy.policynumber) as `gwp_new_policy_average_gross_premium`
        FROM fingl,policy,productsetup
        WHERE ( policy.policynumber = fingl.policynumber AND  
        fingl.currenthistoryflag = 'CURRENT' AND  
        policy.currenthistoryflag = 'CURRENT')  AND 
        (policy.cancellationdate is null OR DATE(policy.cancellationdate)>'{year}/{month}/{ldm}') AND 
        (DATE(policy.inceptiondate)>='{year}/{month}/01' AND DATE(policy.inceptiondate)<='{year}/{month}/{ldm}') AND
        (fingl.type='Premium Raised' AND 
        fingl.posteddate BETWEEN '{year}/{month}/01' AND '{year}/{month}/{ldm}') 
        group by `umaproductsetupid`,`grouptype`,`period`    
    """.format(year=year,month=str(month).zfill(2),ldm=ldm,y=y,m=m,d=d)

In [ ]:
def gwp_new_policies_query_fn(iterate_months):
    union_str=' UNION '
    sqlquery=''
    for iterate_month in iterate_months:
#         print(iterate_month)
        new_policies_query=gwp_new_policies_fn(iterate_month)
        if iterate_month!=iterate_months[-1]:
            sqlquery=sqlquery+new_policies_query+union_str
        else:
            sqlquery=sqlquery+new_policies_query
    return sqlquery

In [ ]:
# print((iterate_months))
spam=pyperclip.copy(gwp_new_policies_query_fn(iterate_months))

### Cancellations
#### cancellations_20210308

In [ ]:
def cancellations_fn(period):
    year=period.year
    month=period.month
    ldm=calendar.monthrange(year, month)[1]
    today = datetime.today()
    d=str(today.day).zfill(2)
#     m=str(today.month).zfill(2)
#     m = calendar.month_name[today.month]
    m=str(today.month).zfill(2)
    y=today.year   
    return """SELECT  '{y}/{m}/01' AS `period_py`
    ,left(policy.cancellationdate,7) AS `period`
    ,'cancellations_{y}{m}{d}' AS `operation`        
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,productsetup.productsetupname AS `product`    
    ,IF (policy.policygroupholdingsid ='' OR 
        (SELECT policygroupholdings.policygroupholdingsname            
        FROM policygroupholdings            
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%indiv%' 
        OR 
        (SELECT policygroupholdings.policygroupholdingsname 
        FROM policygroupholdings 
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%Private%','Individual','Corporate') as `grouptype`
,sum(policy.totalpremium) AS `policies_cancelled_totalpremium`
,count(distinct policy.policynumber) as `policies_cancelled_count`
from policy,productsetup 
  where policy.cancellationdate>policy.inceptiondate AND 
  (DATE(policy.cancellationdate)>='{year}/{month}/01' AND DATE(policy.cancellationdate)<='{year}/{month}/{ldm}') AND
  policy.currenthistoryflag = 'CURRENT'
  group by `umaproductsetupid`,`grouptype`,`period`
    UNION
SELECT
'{year}/{m}/01' AS `period_py`
,left(policy.cancellationdate,7) AS `period`
,'cancellations_{y}{m}{d}' AS `operation`        
,policy.umaproductsetupid AS `umaproductsetupid`
,productsetup.productsetupname AS `product`    
,'aggregate' as `grouptype` 
,sum(policy.totalpremium) AS `policies_cancelled_totalpremium`
,count(distinct policy.policynumber) as `policies_cancelled_count`
from policy,productsetup 
  where policy.cancellationdate>policy.inceptiondate AND 
  (DATE(policy.cancellationdate)>='{year}/{month}/01' AND DATE(policy.cancellationdate)<='{year}/{month}/{ldm}') AND
  policy.currenthistoryflag = 'CURRENT'
  group by `umaproductsetupid`,`grouptype`,`period`
  """.format(year=year,month=month, ldm=ldm,y=y,m=m,d=d)

In [ ]:
def cancellations_query_fn(iterate_months):
    union_str=' UNION '
    sqlquery=''
    for iterate_month in iterate_months:
#         print(iterate_month)
        new_policies_query=cancellations_fn(iterate_month)
        if iterate_month!=iterate_months[-1]:
            sqlquery=sqlquery+new_policies_query+union_str
        else:
            sqlquery=sqlquery+new_policies_query
    return sqlquery

In [ ]:
period=iterate_months[-1]

In [ ]:
year=period.year
month=period.month
ldm=calendar.monthrange(year, month)[1]
today = datetime.today()
d=str(today.day).zfill(2)
#     m=str(today.month).zfill(2)
#     m = calendar.month_name[today.month]
m=str(today.month).zfill(2)
y=today.year   


In [ ]:
# print(cancellations_query_fn(iterate_months))
spam=pyperclip.copy(cancellations_query_fn(iterate_months))

### NTU
#### ntu_20210308

In [ ]:
def ntu_fn(period):
    year=period.year
    month=period.month
    ldm=calendar.monthrange(year, month)[1]
    today = datetime.today()
    d=str(today.day).zfill(2)
#     m=str(today.month).zfill(2)
#     m = calendar.month_name[today.month]
    m=str(today.month).zfill(2)
    y=today.year   
    return """
    SELECT '{year}/{month}/01' as `period_py`
    ,'ntu_policies_{y}{m}{d}' AS `operation` 
    ,left(policy.updateddate,7) AS `period` 
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,productsetup.productsetupname AS `product`    
	,IF(policy.policygroupholdingsid ='' OR (SELECT policygroupholdings.policygroupholdingsname
											FROM policygroupholdings
											WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid
											AND policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')) LIKE '%indiv%' 
	   									OR (SELECT policygroupholdings.policygroupholdingsname
											FROM policygroupholdings
											WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid
											AND policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')) LIKE '%Private%' 
	   ,'individual','corporate') as `grouptype`
       ,count(distinct policy.policynumber) AS ntu_policy_count      
	   ,sum(totalpremium) AS ntu_policy_premium
       ,sum(totalpremium)/count(distinct policy.policynumber) AS ntu_policy_average_premium
from policy,productsetup
where policy.status='Not Taken Up'
and policy.policytransactionid = (SELECT p.policytransactionid
								  FROM policy p
								  WHERE p.policynumber=policy.policynumber
								  AND DATE(p.updateddate) BETWEEN '{year}/{month}/01' AND '{year}/{month}/{ldm}'
								  ORDER by p.updateddate DESC
								  LIMIT 1)
  group by `umaproductsetupid`,`grouptype`,`period`
UNION
    SELECT '{year}/{month}/01' as `period_py`
    ,'ntu_policies_{y}{m}{d}' AS `operation` 
    ,left(policy.updateddate,7) AS `period` 
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,productsetup.productsetupname AS `product`    
	,'aggregate' as `grouptype`
       ,count(distinct policy.policynumber) AS ntu_policy_count      
	   ,sum(totalpremium) AS ntu_policy_premium
       ,sum(totalpremium)/count(distinct policy.policynumber) AS ntu_policy_average_premium
from policy,productsetup
where policy.status='Not Taken Up'
and policy.policytransactionid = (SELECT p.policytransactionid
								  FROM policy p
								  WHERE p.policynumber=policy.policynumber
								  AND DATE(p.updateddate) BETWEEN '{year}/{month}/01' AND '{year}/{month}/{ldm}'
								  ORDER by p.updateddate DESC
								  LIMIT 1)
  group by `umaproductsetupid`,`grouptype`,`period`
  """.format(year=year,month=month, ldm=ldm,y=y,m=m,d=d)

In [ ]:
def ntu_query_fn(iterate_months):
    union_str=' UNION '
    sqlquery=''
    for iterate_month in iterate_months:
#         print(iterate_month)
        new_policies_query=ntu_fn(iterate_month)
        if iterate_month!=iterate_months[-1]:
            sqlquery=sqlquery+new_policies_query+union_str
        else:
            sqlquery=sqlquery+new_policies_query
    return sqlquery

In [ ]:
# print(cancellations_query_fn(iterate_months))
spam=pyperclip.copy(ntu_query_fn(iterate_months))

## Claims

#### under claims, four queries are calculated:
1. claims received
    - claims_received_count
    - claims_received_value
    - claims_received_average_value
2. repudiated claims
3. outstanding claims

### claims recieved
#### claims_recieved_20210308

In [ ]:
def claims_received_fn(iterate_periods):
    start_month=str(min(iterate_periods).month).zfill(2)
    start_year=str(min(iterate_periods).year).zfill(4)
    end_month=str(max(iterate_periods).month).zfill(2)
    end_year=str(max(iterate_periods).year).zfill(4)
    end_day=str(calendar.monthrange(max(iterate_periods).year, max(iterate_periods).month)[1]).zfill(2)
    today = datetime.today()
    today_day=str(today.day).zfill(2)
#     m=str(today.month).zfill(2)
#     m = calendar.month_name[today.month]
    today_month=str(today.month).zfill(2)
    today_year=str(today.year).zfill(4) 
    return """SELECT left(DATE_FORMAT(claim.updateddate,'%Y-%m-%d'),7) AS `period`
    ,'{today_year}{today_month}{today_day}' AS `operation`
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,productsetup.productsetupname AS `product`
    ,claim.umaid AS `umaid`
    ,IF (policy.policygroupholdingsid ='' OR 
        (SELECT policygroupholdings.policygroupholdingsname            
        FROM policygroupholdings            
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%indiv%' 
        OR 
        (SELECT policygroupholdings.policygroupholdingsname 
        FROM policygroupholdings 
        WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
        policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
        ) LIKE '%Private%','Individual','Corporate') AS `grouptype`
    ,COUNT(DISTINCT claim.claimnumber) AS `claims_received_count`
    ,SUM(claim.reserve) AS `claims_received_value`
    ,SUM(claim.reserve)/COUNT(DISTINCT claim.claimnumber) AS `claims_received_average_value`
    from claim,policy,policygroup,client,policygroupholdings,claiminsureditem,productsetup
    where 1=1
    AND DATE_FORMAT(claim.updateddate,'%Y/%m/%d') BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{end_day}'
    group by `period`,`product`,`umaid`,`grouptype`
    UNION
    SELECT left(claim.updateddate,7) AS `period` 
    ,'claims_received_{today_year}{today_month}{today_day}' AS `operation`
    ,policy.umaproductsetupid AS `umaproductsetupid`
    ,productsetup.productsetupname AS `product`
    ,claim.umaid AS `umaid`
    ,'Individual plus Corporate' AS `grouptype`
    ,COUNT(DISTINCT claim.claimnumber) AS `claims_received_count`
    ,SUM(claim.reserve) AS `claims_received_value`
    ,SUM(claim.reserve)/COUNT(DISTINCT claim.claimnumber) AS `claims_received_average_value`
    from claim,policy,policygroup,client,policygroupholdings,claiminsureditem,productsetup
    where 1=1
    AND DATE_FORMAT(claim.updateddate,'%Y/%m/%d') BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{end_day}'
    group by `period`,`product`,`umaid`,`grouptype`
  """.format(start_year=start_year,
             start_month=start_month, 
             end_year=end_year,
             end_month=end_month,
             end_day=end_day,
             today_year=today_year,
             today_month=today_month,
             today_day=today_day)

In [ ]:
spam=pyperclip.copy(claims_received_fn(iterate_months))

### repudiated claims

repudiated_claims_20210308

1. claims_rejected_count
2. claims_rejected_value
3. claims_rejected_average_value

In [ ]:
def repudiated_cliams_fn(iterate_periods):
    start_month=str(min(iterate_periods).month).zfill(2)
    start_year=str(min(iterate_periods).year).zfill(4)
    end_month=str(max(iterate_periods).month).zfill(2)
    end_year=str(max(iterate_periods).year).zfill(4)
    end_day=str(calendar.monthrange(max(iterate_periods).year, max(iterate_periods).month)[1]).zfill(2)
    today = datetime.today()
    today_day=str(today.day).zfill(2)
#     m=str(today.month).zfill(2)
#     m = calendar.month_name[today.month]
    today_month=str(today.month).zfill(2)
    today_year=str(today.year).zfill(4) 
    return """
            SELECT left(DATE_FORMAT(claim.updateddate,'%Y-%m-%d'),7) AS `period`
            ,'{today_year}{today_month}{today_day}' AS `operation`             
            ,policy.umaproductsetupid AS `umaproductsetupid`
            ,productsetup.productsetupname AS `product`
            ,claim.umaid AS `umaid`
            ,IF (policy.policygroupholdingsid ='' OR 
                (SELECT policygroupholdings.policygroupholdingsname            
                FROM policygroupholdings            
                WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
                policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
                ) LIKE '%indiv%' 
                OR 
                (SELECT policygroupholdings.policygroupholdingsname 
                FROM policygroupholdings 
                WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
                policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
                ) LIKE '%Private%','Individual','Corporate') AS `grouptype`
            ,COUNT(DISTINCT claim.claimnumber) AS `claims_rejected_count`
            ,SUM(claim.reserve) AS `claims_rejected_value`
            ,SUM(claim.reserve)/COUNT(DISTINCT claim.claimnumber) AS `claims_rejected_average_value`
            from claim,policy,policygroup,client,policygroupholdings,claiminsureditem,productsetup
            where 1=1 AND
            DATE_FORMAT(claim.updateddate,'%Y/%m/%d') BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{end_day}' AND
            claim.decisionstatus='Reject'
            group by `period`,`product`,`umaid`,`grouptype`
            UNION
            SELECT left(claim.updateddate,7) AS `period`
            ,'repudiated_claims_{today_year}{today_month}{today_day}' AS `operation`             
            ,policy.umaproductsetupid AS `umaproductsetupid`
            ,productsetup.productsetupname AS `product`
            ,claim.umaid AS `umaid`
            ,'Individual plus Corporate' AS `grouptype`
            ,COUNT(DISTINCT claim.claimnumber) AS `claims_rejected_count`
            ,SUM(claim.reserve) AS `claims_rejected_value`
            ,SUM(claim.reserve)/COUNT(DISTINCT claim.claimnumber) AS `claims_rejected_average_value`
            from claim,policy,policygroup,client,policygroupholdings,claiminsureditem,productsetup
            where 1=1 AND
            DATE_FORMAT(claim.updateddate,'%Y/%m/%d') BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{end_day}' AND
            claim.decisionstatus='Reject'
            group by `period`,`product`,`umaid`,`grouptype`
            
  """.format(start_year=start_year,
             start_month=start_month, 
             end_year=end_year,
             end_month=end_month,
             end_day=end_day,
             today_year=today_year,
             today_month=today_month,
             today_day=today_day)

In [ ]:
spam=pyperclip.copy(repudiated_cliams_fn(iterate_months))

### outstanding claims

#### Outstanding_claims_20210308

In [ ]:
def outstanding_claims_fn(iterate_periods):
    start_month=str(min(iterate_periods).month).zfill(2)
    start_year=str(min(iterate_periods).year).zfill(4)
    end_month=str(max(iterate_periods).month).zfill(2)
    end_year=str(max(iterate_periods).year).zfill(4)
    end_day=str(calendar.monthrange(max(iterate_periods).year, max(iterate_periods).month)[1]).zfill(2)
    today = datetime.today()
    today_day=str(today.day).zfill(2)
#     m=str(today.month).zfill(2)
#     m = calendar.month_name[today.month]
    today_month=str(today.month).zfill(2)
    today_year=str(today.year).zfill(4) 
    return """
        SELECT left(claim.updateddate,7) AS `period`
        ,'outstanding_claims_{today_year}{today_month}{today_day}' AS `operation`         
        ,policy.umaproductsetupid AS `umaproductsetupid`
        ,productsetup.productsetupname AS `product`
        ,claim.umaid AS `umaid`
        ,IF (policy.policygroupholdingsid ='' OR 
            (SELECT policygroupholdings.policygroupholdingsname            
            FROM policygroupholdings            
            WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
            policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
            ) LIKE '%indiv%' 
            OR 
            (SELECT policygroupholdings.policygroupholdingsname 
            FROM policygroupholdings 
            WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
            policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
            ) LIKE '%Private%','Individual','Corporate') AS `grouptype`
        ,COUNT(DISTINCT claim.claimnumber) AS `claims_outstanding_count`
        ,SUM(claiminsureditem.reserve) AS `claims_outstanding_value`
        ,SUM(claiminsureditem.reserve)/COUNT(DISTINCT claim.claimnumber) AS `claims_outstanding_average_value`
        from claim,policy,policygroup,client,policygroupholdings,claiminsureditem,productsetup
        where 1=1 AND
        DATE_FORMAT(claim.updateddate,'%Y/%m/%d') BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{end_day}' AND
        claim.manualstatus='Outstanding docs received - sent to processing'
        GROUP BY `period`,`product`,`umaid`,`grouptype`
        UNION
        SELECT left(claim.updateddate,7) AS `period`
        ,'outstanding_claims_{today_year}{today_month}{today_day}' AS `operation`         
        ,policy.umaproductsetupid AS `umaproductsetupid`
        ,productsetup.productsetupname AS `product`
        ,claim.umaid AS `umaid`
        ,'Individual plus Corporate' AS `grouptype`
        ,COUNT(DISTINCT claim.claimnumber) AS `claims_outstanding_count`
        ,SUM(claiminsureditem.reserve) AS `claims_outstanding_value`
        ,SUM(claiminsureditem.reserve)/COUNT(DISTINCT claim.claimnumber) AS `claims_outstanding_average_value`
        from claim,policy,policygroup,client,policygroupholdings,claiminsureditem,productsetup
        where 1=1 AND
        DATE_FORMAT(claim.updateddate,'%Y/%m/%d') BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{end_day}' AND
        claim.manualstatus='Outstanding docs received - sent to processing'
        GROUP BY `period`,`product`,`umaid`,`grouptype`
        
  """.format(start_year=start_year,
             start_month=start_month, 
             end_year=end_year,
             end_month=end_month,
             end_day=end_day,
             today_year=today_year,
             today_month=today_month,
             today_day=today_day)

In [ ]:
spam=pyperclip.copy(outstanding_claims_fn(iterate_months))

### claims paid

#### paid_claims_20210308

In [ ]:
def paid_claims_fn(iterate_periods):
    start_month=str(min(iterate_periods).month).zfill(2)
    start_year=str(min(iterate_periods).year).zfill(4)
    end_month=str(max(iterate_periods).month).zfill(2)
    end_year=str(max(iterate_periods).year).zfill(4)
    end_day=str(calendar.monthrange(max(iterate_periods).year, max(iterate_periods).month)[1]).zfill(2)
    today = datetime.today()
    today_day=str(today.day).zfill(2)
#     m=str(today.month).zfill(2)
#     m = calendar.month_name[today.month]
    today_month=str(today.month).zfill(2)
    today_year=str(today.year).zfill(4) 
    return """
        select left(DATE_FORMAT(claim.updateddate,'%Y-%m-%d'),7) AS `period`
        ,'{today_year}{today_month}{today_day}' AS `operation`              
        ,policy.umaproductsetupid AS `umaproductsetupid`
        ,productsetup.productsetupname AS `product`
        ,claim.umaid AS `umaid`
        ,IF (policy.policygroupholdingsid ='' OR 
            (SELECT policygroupholdings.policygroupholdingsname            
            FROM policygroupholdings            
            WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
            policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
            ) LIKE '%indiv%' 
            OR 
            (SELECT policygroupholdings.policygroupholdingsname 
            FROM policygroupholdings 
            WHERE policygroupholdings.policygroupholdingsid=policy.policygroupholdingsid AND 
            policygroupholdings.currenthistoryflag=CONCAT('C','URRENT')
            ) LIKE '%Private%','Individual','Corporate') AS `grouptype`
        ,MAX(claim.payout) AS `claims_paid_maximum`
        ,COUNT(DISTINCT claim.claimnumber) AS `claims_paid_count`
        ,SUM(claim.payout) AS `claims_paid_amount`        
        ,SUM(claim.reserve) AS `claims_paid_reserve`
        ,SUM(claiminsureditem.reserve) AS `claims_paid_claiminsureditem_reserve`
        from claim,policy,policygroup,client,policygroupholdings,claiminsureditem,productsetup
        where 1=1 AND
        DATE_FORMAT(claim.updateddate,'%Y/%m/%d') BETWEEN '{start_year}/{start_month}/01' AND '{end_year}/{end_month}/{end_day}' AND
        claim.decisionstatus='Approve, Pay Now'
        group by `period`,`product`,`umaid`,`grouptype`
        
  """.format(start_year=start_year,
             start_month=start_month, 
             end_year=end_year,
             end_month=end_month,
             end_day=end_day,
             today_year=today_year,
             today_month=today_month,
             today_day=today_day)

In [ ]:
spam=pyperclip.copy(paid_claims_fn(iterate_months))

#### Create Product List

In [ ]:
claims_received_query=claims_received_fn(iterate_months)
repudiated_cliams_query=repudiated_cliams_fn(iterate_months)

In [ ]:
spam=pyperclip.copy(repudiated_cliams_query)

In [ ]:
# period,
# operation
# umaproductsetupid
# product
# umaid

In [ ]:
sqlquery="""
SELECT claims_received.* FROM 
    (
        {claims_received_query}
    ) AS claims_received
INNER JOIN 
    (
        {repudiated_cliams_query}
    ) AS repudiated_cliams
ON 
claims_received.period = repudiated_cliams.period AND
claims_received.umaproductsetupid = repudiated_cliams.umaproductsetupid AND
claims_received.product = repudiated_cliams.product AND
claims_received.umaid = repudiated_cliams.umaid
claims_received.operation = repudiated_cliams.operation
""".format(claims_received_query=claims_received_query,
           repudiated_cliams_query=repudiated_cliams_query)

In [ ]:
spam=pyperclip.copy(sqlquery)

In [ ]:
f=~owls_terminations_df['productsetup_productsetupname'].isna()
product_list=list(set(owls_terminations_df[f]['productsetup_productsetupname']))

In [ ]:
members_live_filter=(owls_members_df['policy_status']=='Live')
ntu_filter=owls_terminations_df['policy_status']=='Not Taken Up'
cancelations_filter=owls_terminations_df['policy_status']=='Cancelled'
# owls_members_live_filter=owls_members_df['policy_status']=='Live'
# Received_Claims! 𝐾: K,"Reject"
# Received_Claims!$K:$K,"Approve, Pay now")

In [ ]:
# owls_received_claims_df[outstanding_claims_filter]

### Corporate Individual Split

In [ ]:
# owls_members_df['policy_paymentmethod']
owls_members_df=owls_members_df[['policy_policynumber','policygroup_policygroupname','policy_paymentmethod','policy_grosspremium','policy_status','policy_totalpremium','productsetup_productsetupname','policy_inceptiondate']].copy()
owls_terminations_df=owls_terminations_df[['policy_policynumber','policy_paymentmethod','policy_totalpremium','productsetup_productsetupname','policy_inceptiondate','policy_cancellationdate']].copy()
owls_received_claims_df=owls_received_claims_df[['Claim Number','Total Payments','Notification Date','productsetupname','som_notification_date','Original Reserve','notification_date','Progress','Status']].copy()

In [ ]:
owls_members_df['corporate_individual']=''
f=owls_members_df['policy_paymentmethod']=='Debit'
g=(owls_members_df['policy_paymentmethod']!='Debit')&(owls_members_df['policy_paymentmethod']!='')
owls_members_df.loc[f, 'corporate_individual_split']=owls_members_df['productsetup_productsetupname'] + ' - Individual'
owls_members_df.loc[g, 'corporate_individual_split']=owls_members_df['productsetup_productsetupname'] + ' - Corporate'

In [ ]:
owls_terminations_df['corporate_individual']=''
f=owls_terminations_df['policy_paymentmethod']=='Debit'
g=(owls_terminations_df['policy_paymentmethod']!='Debit')&(owls_terminations_df['policy_paymentmethod']!='')
owls_terminations_df.loc[f, 'corporate_individual_split']=owls_terminations_df['productsetup_productsetupname'] + ' - Individual'
owls_terminations_df.loc[g, 'corporate_individual_split']=owls_terminations_df['productsetup_productsetupname'] + ' - Corporate'

In [ ]:
owls_received_claims_df[['policy_policynumber_a','policy_policynumber_b']]=owls_received_claims_df['Claim Number'].str.split("/",expand = True)
owls_received_claims_df.policy_policynumber_a = owls_received_claims_df.policy_policynumber_a.str.strip()
df1=owls_received_claims_df.merge(owls_members_df[['policy_policynumber','corporate_individual_split']], left_on= 'policy_policynumber_a', right_on='policy_policynumber', how='inner')
df2=owls_received_claims_df.merge(owls_terminations_df[['policy_policynumber','corporate_individual_split']], left_on= 'policy_policynumber_a', right_on='policy_policynumber', how='inner')
gf=df1.append(df2, ignore_index = True) 

In [ ]:
owls_received_claims_df=gf.copy()
pay_now_filter=owls_received_claims_df['Progress']=='Approve, Pay Now'
reject_claims_filter=owls_received_claims_df['Progress']=='Reject'
outstanding_claims_filter=owls_received_claims_df['Status']=='Outstanding docs received - sent to processing'

In [ ]:
product_list=list(set(owls_members_df['corporate_individual_split']))

In [ ]:
product_list = [x for x in product_list if str(x) != 'nan']

In [ ]:
product_list

### Kaelo Gap

In [ ]:
owls_terminations_temp_df=owls_terminations_df.copy()

In [ ]:
product='Sanlam Gap - Individual'

In [ ]:
max_date=max([owls_members_df['policy_inceptiondate'].max(),owls_terminations_df['policy_inceptiondate'].max(),owls_terminations_df['policy_cancellationdate'].max()])
# policy_inceptiondate,policy_cancellationdate

In [ ]:
owls_members_df=owls_members_df[['policy_policynumber','policy_grosspremium','policy_status','policy_totalpremium','productsetup_productsetupname','policy_inceptiondate','corporate_individual_split']].copy()
owls_terminations_df=owls_terminations_df[['policy_totalpremium','productsetup_productsetupname','policy_inceptiondate','policy_cancellationdate','corporate_individual_split']].copy()
owls_received_claims_df=owls_received_claims_df[['Total Payments','Notification Date','productsetupname','som_notification_date','Original Reserve','notification_date','corporate_individual_split']].copy()

In [ ]:
terminations_product_filter=owls_terminations_df['corporate_individual_split']==product
members_product_filter=(owls_members_df['corporate_individual_split']==product)
claims_received_product_filter=(owls_received_claims_df['corporate_individual_split']==product)
# product_filter=(owls_members_df['productsetup_productsetupname']==product)
# live_filter=(owls_members_df['policy_status']=='Live')

Count live policies and calculate gross premium:

In [ ]:
live_df=owls_members_df[members_live_filter&members_product_filter]
slice_saver(identifier_name="live_policies", dataframe=live_df, save_path=slices_path, datafile_name=owls_members, product_name=product)

live_policies=live_df.shape[0]
live_df=owls_members_df[members_live_filter&members_product_filter]
live_policies_grosspremium=live_df['policy_totalpremium'].sum()

Active Policies

There are three components to counting policies.
 - count only active policies
 


 - COUNTIFS(Active_Listing_Raw!$I:$I,"Live",Active_Listing_Raw!$D:$D,Summary!$C$2)- 
 - COUNTIFS(Active_Listing_Raw!$G:$G,">"&Summary!AC5,Active_Listing_Raw!$D:$D,Summary!$C$2)+
 - COUNTIFS(Terminations_Raw!$AF:$AF,">"&Summary!AD5, #
 - Terminations_Raw!$AG:$AG,">"&Summary!AB5,Terminations_Raw!$D:$D,Summary!$C$2)

In [ ]:
print('COUNTIFS(Terminations_Raw!$AF:$AF,"<"&Summary!AD5,Terminations_Raw!$AG:$AG,">"&Summary!AB5,Active_Listing_Raw!$D:$D,Summary!$C$2)')

 - Terminations_Raw!$AF:$AF,"<"&Summary!AD5 : all policies with inception date () __BEFORE__ the start of the next month __and__
 - Terminations_Raw!$AG:$AG,">"&Summary!AB5 : all policies with termination date __AFTER__ the start of the previous month __and__
 - Active_Listing_Raw!$D:$D,Summary!$C$2 : policies of this product

### Active Policies

***
The number of __Active policies__ contain three items. For those policies of the specified product:
 - __Live Policies__: Count "live" policies only, that is where policy_status='live', this covers all data. This number is constant for all periods as it is only calculated on the latest complete data set and is not done on a by period basis.  To calculate for a particular period the dataset adjusted by doing the following:
 - __New Inactive Policies__: (Subtract) all policies that have inception date from the next period onwards.
 - __Active Terminated Policies__: (Add back) all __terminated__ policies that had inception date (policy_inceptiondate) __BEFORE__ the start of the next period and where the termination date (policy_cancellationdate) is __AFTER__ the start of the previous period
 - The previous period __New Inactive Policies__ MINUS the currrent period __New Inactive Policies__ yield the current period __New Policies__.

In [ ]:

# date_filter=owls_members_df['policy_inceptiondate']>datelist[-3]

In [ ]:
# terminations_policy_inceptiondate_filter=owls_terminations_df['policy_inceptiondate']<iterate_months[i+1]

In [ ]:
# s=owls_members.split("_")[1].split(".")[0]
# filename="cbr"+'_'+product+"_"+s+".csv"
# df.to_csv(path_or_buf=save_path+datafile_name, sep=',', index=True)

s=owls_members.split("_")[1].split(".")[0]
filename="cbr"+'_'+product+"_"+s+".csv"
df.to_csv(path_or_buf=slices_path+filename, sep=',', index=True)

In [ ]:
iterate_months

In [ ]:
# set(owls_members_df['policy_inceptiondate'])

In [ ]:
set(owls_members_df['policy_inceptiondate'])

In [ ]:
owls_members_df[owls_members_df['policy_inceptiondate']==iterate_months[-1]]

In [ ]:
iterate_months

In [ ]:
# terminations_product_filter=owls_terminations_df['productsetup_productsetupname']==product
live_policies_list=[]
new_inactive_policies_liast=[]
active_policies_list=[]
valid_terminations_list=[]
for i in range(1,len(iterate_months)-1):
    print(i,iterate_months[i])

    members_date_filter=owls_members_df['policy_inceptiondate']>iterate_months[i]
    new_policies_df=owls_members_df[members_date_filter&members_product_filter]
    slice_saver(identifier_name="subsequent_new_policies", dataframe=new_policies_df, save_path=slices_path, datafile_name=owls_members, product_name=product)
    
    new_inactive_policies=new_policies_df.shape[0]
    terminations_policy_inceptiondate_filter=owls_terminations_df['policy_inceptiondate']<=max_date #iterate_months[i+1]
    terminations_policy_cancellationdate_filter=owls_terminations_df['policy_cancellationdate']>iterate_months[i-1]
    valid_terminations_filter=(terminations_policy_inceptiondate_filter&terminations_policy_cancellationdate_filter&terminations_product_filter)
    valid_terminations_df=owls_terminations_df[valid_terminations_filter]
    slice_saver(identifier_name="valid_terminations", dataframe=valid_terminations_df, save_path=slices_path, datafile_name=owls_members, product_name=product)
    
    valid_terminations=valid_terminations_df.shape[0]
    active_policies=live_policies-new_inactive_policies+valid_terminations    
    print('i = ',i,'; month: ', iterate_months[i],owls_terminations_df[terminations_policy_cancellationdate_filter].shape[0],'; active_policies: ',active_policies,'; live_policies: ',live_policies,'; new_inactive_policies: ',new_inactive_policies, ' valid_terminations: ',valid_terminations)
    live_policies_list.append(live_policies)
    new_inactive_policies_list.append(new_inactive_policies)
    valid_terminations_list.append(valid_terminations)
    active_policies_list.append(active_policies)

In [ ]:
active_policies_list

### Gross Premium

 - COUNTIFS(Active_Listing_Raw!$I:$I,"Live",Active_Listing_Raw!$D:$D,Summary!$C$2)- 
 - COUNTIFS(Active_Listing_Raw!$G:$G,">"&Summary!AC5,Active_Listing_Raw!$D:$D,Summary!$C$2)+
 - COUNTIFS(Terminations_Raw!$AF:$AF,">"&Summary!AD5, #
 - Terminations_Raw!$AG:$AG,">"&Summary!AB5,Terminations_Raw!$D:$D,Summary!$C$2)

 - SUMIFS(Active_Listing_Raw!$AE:$AE,Active_Listing_Raw!$I:$I,"Live",Active_Listing_Raw!$D:$D,Summary!$C$2)
 - SUMIFS(Active_Listing_Raw!$AE:$AE,Active_Listing_Raw!$G:$G,">"&Summary!X5,Active_Listing_Raw!$D:$D,Summary!$C$2)
 - SUMIFS(Terminations_Raw!$AD:$AD,Terminations_Raw!$AF:$AF,"<"&Summary!Z5,Terminations_Raw!$AG:$AG,">"&Summary!W5,Terminations_Raw!$D:$D,Summary!$C$2)

__ Notice the descrepancy here.  In calculating the number of valid terminated policies the excel spreadsheet conditions on AF  being later than 

In [ ]:
live_policies_grosspremium_list=[]
new_policies_grosspremium_list=[]
active_grosspremium_list=[]
valid_policies_grosspremium_list=[]
for i in range(1,len(iterate_months)-1):
    print(i,iterate_months[i])
    member_date_filter=owls_members_df['policy_inceptiondate']>iterate_months[i]
#   ----------------------------------------------------------------------------------------------------------------------------
    new_policies_df=owls_members_df[member_date_filter&members_product_filter]
    new_policies_grosspremium=new_policies_df['policy_totalpremium'].sum()
#   ----------------------------------------------------------------------------------------------------------------------------
    terminations_policy_inceptiondate_filter=owls_terminations_df['policy_inceptiondate']<=max_date#iterate_months[i+1]
    terminations_policy_cancellationdate_filter=owls_terminations_df['policy_cancellationdate']>iterate_months[i-1]
    valid_terminations_filter=(terminations_policy_inceptiondate_filter&terminations_policy_cancellationdate_filter&terminations_product_filter)
    valid_terminations_df=owls_terminations_df[valid_terminations_filter]
#     slice_saver(identifier_name="valid_terminations", dataframe=new_policies_df, save_path=slices_path, datafile_name=owls_members, product_name=product)    
    valid_policies_grosspremium=valid_terminations_df['policy_totalpremium'].sum()
#   ----------------------------------------------------------------------------------------------------------------------------
    active_policies_grosspremium=live_policies_grosspremium-new_policies_grosspremium+valid_policies_grosspremium    
    print('i = ',i,'; month: ', iterate_months[i],'; iterate_months[i-1]: ',iterate_months[i-1],'; active_policies_grosspremium: ',active_policies_grosspremium,'; live_policies_grosspremium: ',live_policies_grosspremium,'; new_policies_grosspremium: ',new_policies_grosspremium, ' valid_policies_grosspremium: ',valid_policies_grosspremium)
    live_policies_grosspremium_list.append(live_policies_grosspremium)
    new_policies_grosspremium_list.append(new_policies_grosspremium)
    valid_policies_grosspremium_list.append(valid_policies_grosspremium)
    active_grosspremium_list.append(active_policies_grosspremium)

In [ ]:
new_policies_df

In [ ]:
product.lower().replace("-","").replace("  "," ").replace(" ","_")

In [ ]:
# slice_saver(identifier_name="new_policies", dataframe=new_policies_df, save_path=slices_path, datafile_name=owls_members, product_name=product)

In [ ]:
# # terminations_product_filter=owls_terminations_df['productsetup_productsetupname']==product
# live_policies_list=[]
# new_policies_list=[]
# active_policies_list=[]
# valid_terminations_list=[]

# live_policies_grosspremium_list=[]
# new_policies_grosspremium_list=[]
# active_grosspremium_list=[]
# valid_policies_grosspremium_list=[]

# for i in range(1,len(iterate_months)-1):
#     print(i,iterate_months[i])

#     members_date_filter=owls_members_df['policy_inceptiondate']>iterate_months[i]
#     new_policies_df=owls_members_df[members_date_filter&members_product_filter]
    
#     new_policies=new_policies_df.shape[0]
#     new_policies_grosspremium=new_policies_df['policy_totalpremium'].sum()
    
#     terminations_policy_inceptiondate_filter=owls_terminations_df['policy_inceptiondate']<iterate_months[i+1]
#     terminations_policy_cancellationdate_filter=owls_terminations_df['policy_cancellationdate']>iterate_months[i-1]
    
#     valid_terminations_filter=(terminations_policy_inceptiondate_filter&terminations_policy_cancellationdate_filter&terminations_product_filter)
#     valid_terminations_df=owls_terminations_df[valid_terminations_filter]
    
#     valid_terminations=valid_terminations_df.shape[0]
#     valid_policies_grosspremium=valid_terminations_df['policy_totalpremium'].sum()
    
#     active_policies=live_policies-new_policies+valid_terminations 
#     active_policies_grosspremium=live_policies_grosspremium-new_policies_grosspremium+valid_policies_grosspremium    
    
#     live_policies_list.append(live_policies)
#     new_policies_list.append(new_policies)
#     valid_terminations_list.append(valid_terminations)
#     active_policies_list.append(active_policies)
    
#     live_policies_grosspremium_list.append(live_policies_grosspremium)
#     new_policies_grosspremium_list.append(new_policies_grosspremium)
#     valid_policies_grosspremium_list.append(valid_policies_grosspremium)
#     active_grosspremium_list.append(active_policies_grosspremium)    
    
#     print('i = ',i,'; month: ', iterate_months[i],owls_terminations_df[terminations_policy_cancellationdate_filter].shape[0],'; active_policies: ',active_policies,'; live_policies: ',live_policies,'; new_policies: ',new_policies, ' valid_terminations: ',valid_terminations)


### New policies

In [ ]:
# members_product_filter=(owls_members_df['productsetup_productsetupname']==product)
# new_policy_filter=owls_members_df['policy_inceptiondate']==
# owls_members_df[date_filter&product_filter].shape[0]

Add back recent terminations falling in the window still to be deemed valid:
- =COUNTIFS(Terminations_Raw!$AF:$AF,"<"&Summary!AD5,Terminations_Raw!$AG:$AG,">"&Summary!AB5,Active_Listing_Raw!$D:$D,Summary!$C$2)

 - =COUNTIFS(Active_Listing_Raw!$D:$D,Summary!$C$2,Active_Listing_Raw!$F:$F,Summary!AC5)

pd.to_datetime(owls_terminations_df['policy_inceptiondate'], format='%Y/%m/%d', errors='ignore')

In [ ]:
# members_product_filter=(owls_members_df['productsetup_productsetupname']==product)
new_policies_list=[]
for i in range(1,len(iterate_months)-1):
    print(i,iterate_months[i], iterate_months[i+1])
# 
#     member_date_filter=owls_members_df['policy_inceptiondate']>iterate_months[i]
#     new_policies_df=owls_members_df[member_date_filter&members_product_filter]
# 
    
    new_policies_date_filter=(owls_members_df['policy_inceptiondate']>=iterate_months[i])&
    (owls_members_df['policy_inceptiondate']<iterate_months[i+1])
#   ----------------------------------------------------------------------------------------------------------------------------
    new_policies_df=owls_members_df[new_policies_date_filter&members_product_filter]
    slice_saver(identifier_name="period_new_policies", dataframe=new_policies_df, save_path=slices_path, datafile_name=owls_members, product_name=product)
    
    new_policies=new_policies_df['policy_totalpremium'].shape[0]
    new_policies_list.append(new_policies)

In [ ]:
new_policies_list

### GWP of New Policies

=SUMIFS(Active_Listing_Raw!$AC:$AC,Active_Listing_Raw!$D:$D,Summary!$C$2,Active_Listing_Raw!$F:$F,Summary!X5)

In [ ]:
# members_product_filter=(owls_members_df['productsetup_productsetupname']==product)
gwp_new_policies_list=[]
for i in range(1,len(iterate_months)-1):
#     print(i,iterate_months[i])
    new_policies_date_filter=owls_members_df['policy_inceptiondate']==iterate_months[i]
#   ----------------------------------------------------------------------------------------------------------------------------
    new_policies_df=owls_members_df[new_policies_date_filter&members_product_filter]
    
    gwp_new_policies=new_policies_df['policy_grosspremium'].sum()
    gwp_new_policies_list.append(gwp_new_policies)
    print(i,iterate_months[i],'; ', gwp_new_policies)

### Average Premium

The average premium is the gross premium(from active_grosspremium_list) __divided__ by the number of active policies (from active_policies_list): 

In [ ]:
active_grosspremium_list
active_policies_list
average_premium_list=[]
for f, b in zip(active_grosspremium_list, active_policies_list):
    average_premium_list.append(f/b)
    print(f,b,f/b)

### NTU

=COUNTIFS(Terminations_Raw!$AF:$AF,Summary!Y$5,Terminations_Raw!$H:$H,"Not Taken Up",Terminations_Raw!$D:$D,Summary!$C$2)

In [ ]:
# terminations_product_filter=owls_terminations_df['productsetup_productsetupname']==product
ntu_list=[]
for i in range(1,len(iterate_months)-1):
    terminations_policy_inceptiondate_filter=owls_terminations_df['policy_inceptiondate']==iterate_months[i]
    f=terminations_product_filter&ntu_filter&terminations_policy_inceptiondate_filter
    valid_terminations_df=owls_terminations_df[f]
    ntu=valid_terminations_df.shape[0]
    ntu_list.append(ntu)
    print(i,iterate_months[i],'; ', ntu)

### Gross premium for NTU

=SUMIFS(Terminations_Raw!$AD:$AD,Terminations_Raw!$AF:$AF,Summary!X$5,Terminations_Raw!$H:$H,"Not Taken Up",Terminations_Raw!$D:$D,Summary!$C$2)

In [ ]:
# owls_terminations_df
# terminations_product_filter
# ntu_filter=owls_terminations_df['policy_status']=='Not Taken Up'
# terminations_policy_inceptiondate_filter=owls_terminations_df['policy_inceptiondate']==iterate_months[i]
# sum over policy_totalpremium
# 'policy_inceptiondate'

In [ ]:
# ntu_filter=owls_terminations_df['policy_status']=='Not Taken Up'
gp_ntu_list=[]
for i in range(1,len(iterate_months)-1):
    terminations_policy_inceptiondate_filter=owls_terminations_df['policy_inceptiondate']==iterate_months[i]
    f=terminations_product_filter&ntu_filter&terminations_policy_inceptiondate_filter
    valid_terminations_df=owls_terminations_df[f]
    gp_ntu=valid_terminations_df['policy_totalpremium'].sum()
    gp_ntu_list.append(gp_ntu)
    print(i,iterate_months[i],'; ', gp_ntu,owls_terminations_df[terminations_policy_inceptiondate_filter].shape[0])

***

### Cancellations

=COUNTIFS(Terminations_Raw!$AG:$AG,Summary!X$4,Terminations_Raw!$H:$H,"Cancelled",Terminations_Raw!$D:$D,Summary!$C$2)  
policy_cancellationdate,cancelations_filter,terminations_product_filter
'policy_totalpremium'

In [ ]:
cancelations_list=[]
gp_cancelations_list=[]
for i in range(1,len(iterate_months)-1):
    last_day_of_month=last_day_of_month_fn(iterate_months[i])
    terminations_policy_cancellationdate_filter=owls_terminations_df['policy_cancellationdate']==last_day_of_month
    f=terminations_product_filter&cancelations_filter&terminations_policy_cancellationdate_filter
    valid_terminations_df=owls_terminations_df[f]
    cancelations=valid_terminations_df.shape[0]
    cancelations_list.append(cancelations)
    gp_cancelations=valid_terminations_df['policy_totalpremium'].sum()
    gp_cancelations_list.append(gp_cancelations)    
    print(i,iterate_months[i],'; ',cancelations, gp_cancelations)

*** 

### Number of Claims Received (Incl O/Docs Claims)

=COUNTIFS(Received_Claims!$H:$H,Summary!$C$2,Received_Claims!$P:$P,Summary!X5)    
Received_Claims! 𝑃: P,Summary!X5: Received Date, 'notification_date' compare notification date with period date  
Received_Claims! 𝐻: H,Summary! 𝐶 2: product filter=received_claims_product_filter compare productsetupname with product 


In [ ]:
claims_received_list=[]
value_claims_received_list=[]
# gp_cancelations_list=[]
for i in range(1,len(iterate_months)-1):
#     last_day_of_month=last_day_of_month_fn(iterate_months[i])
    claims_received_notification_date_filter=owls_received_claims_df['som_notification_date']==iterate_months[i]
    f=claims_received_product_filter&claims_received_notification_date_filter
    df=owls_received_claims_df[f].copy()
    claims_received=df.shape[0]
    claims_received_list.append(claims_received)
    value_claims_received=df['Original Reserve'].sum()
    value_claims_received_list.append(value_claims_received)    
    print(i,iterate_months[i],'; ',claims_received, value_claims_received)

***

### Highest Claim Paid

=MAXIFS(Received_Claims!$L:$L,Received_Claims!$H:$H,Summary!$C$2,Received_Claims!$P:$P,Summary!X5)

In [ ]:
highest_claim_paid_list=[]
# value_claims_received_list=[]
# gp_cancelations_list=[]
for i in range(1,len(iterate_months)-1):
#     last_day_of_month=last_day_of_month_fn(iterate_months[i])
    claims_paid_date_filter=owls_received_claims_df['som_notification_date']==iterate_months[i]
    f=claims_received_product_filter&claims_paid_date_filter    
    df=owls_received_claims_df[f].copy()
    highest_claim_paid=df['Total Payments'].max()
    highest_claim_paid_list.append(highest_claim_paid)    
    print(i,iterate_months[i],'; ',highest_claim_paid)
highest_claim_paid_list=[0 if x != x else x for x in highest_claim_paid_list]

In [ ]:
highest_claim_paid_list

***

### Number, Value and Average Value of Claims Paid

=COUNTIFS(Received_Claims!$H:$H,Summary!$C$2, : this is the product filter
eceived_Claims!$P:$P,Summary!X5, : this is the date filter
Received_Claims!$K:$K,"Approve, Pay now") : pay_now_filter

In [ ]:
number_of_claims_paid_list=[]
value_of_claims_paid_list=[]
average_value_of_claims_paid_list=[]
# value_claims_received_list=[]
# gp_cancelations_list=[]
for i in range(1,len(iterate_months)-1):
#     last_day_of_month=last_day_of_month_fn(iterate_months[i])
    average_value_of_claims_paid=0
    claims_paid_date_filter=owls_received_claims_df['som_notification_date']==iterate_months[i]
    f=claims_received_product_filter&claims_paid_date_filter&pay_now_filter
    df=owls_received_claims_df[f].copy()
    number_of_claims_paid=df.shape[0]
    value_of_claims_paid=df['Original Reserve'].sum()
    number_of_claims_paid_list.append(number_of_claims_paid)
    value_of_claims_paid_list.append(value_of_claims_paid)
    if number_of_claims_paid!=0:
        average_value_of_claims_paid=value_of_claims_paid/number_of_claims_paid
#         average_value_of_claims_paid_list.append(average_value_of_claims_paid)     
    average_value_of_claims_paid_list.append(average_value_of_claims_paid)
    print(i,iterate_months[i],'; ',average_value_of_claims_paid,number_of_claims_paid,value_of_claims_paid)
# Average Cost Per Claim


***

### Number, Value and Average Value of Repudiated Claims

=COUNTIFS(Received_Claims!$H:$H,Summary!$C$2,Received_Claims!$P:$P,Summary!X5,Received_Claims!$K:$K,"Reject")

In [ ]:
# reject_claims_filter
reject_claims_list=[]
value_of_reject_claims_list=[]
average_value_of_reject_claims_list=[]
# average_value_of_claims_paid_list=[]
# value_claims_received_list=[]
# gp_cancelations_list=[]
for i in range(1,len(iterate_months)-1):
#     last_day_of_month=last_day_of_month_fn(iterate_months[i])
    average_value_of_reject_claims=0
    claims_paid_date_filter=owls_received_claims_df['som_notification_date']==iterate_months[i]
    f=claims_received_product_filter&claims_paid_date_filter&reject_claims_filter
    df=owls_received_claims_df[f].copy()
    reject_claims=df.shape[0]
    value_of_reject_claims=df['Original Reserve'].sum()
    reject_claims_list.append(reject_claims)
    value_of_reject_claims_list.append(value_of_reject_claims)
    if reject_claims!=0:
        average_value_of_reject_claims=value_of_reject_claims/reject_claims
#         average_value_of_reject_claims_list.append(average_value_of_reject_claims)     
    average_value_of_reject_claims_list.append(average_value_of_reject_claims)
    print(i,iterate_months[i],'; average_value_of_reject_claims: ',average_value_of_reject_claims,'; reject_claims: ',reject_claims,'; value_of_reject_claims: ',value_of_reject_claims)
# Average Cost Per Claim


***

### Number, Value and Average Value of Outstanding Claims

=COUNTIFS(Received_Claims!$H:$H,Summary!$C$2,Received_Claims!$P:$P,Summary!X5,Received_Claims!$K:$K,"")

In [ ]:
# reject_claims_filter
outstanding_claims_list=[]
value_of_outstanding_claims_list=[]
average_value_of_outstanding_claims_list=[]
# average_value_of_claims_paid_list=[]
# value_claims_received_list=[]
# gp_cancelations_list=[]
for i in range(1,len(iterate_months)-1):
#     last_day_of_month=last_day_of_month_fn(iterate_months[i])
    average_value_of_outstanding_claims=0
    outstanding_claims_date_filter=owls_received_claims_df['som_notification_date']==iterate_months[i]
    f=claims_received_product_filter&outstanding_claims_date_filter&outstanding_claims_filter
    print(owls_received_claims_df[outstanding_claims_filter].shape[0])
    df=owls_received_claims_df[f].copy()
    outstanding_claims=df.shape[0]
    value_of_outstanding_claims=df['Original Reserve'].sum()
    outstanding_claims_list.append(outstanding_claims)
    value_of_outstanding_claims_list.append(value_of_outstanding_claims)
    if outstanding_claims!=0:
        average_value_of_outstanding_claims=value_of_outstanding_claims/outstanding_claims
#         average_value_of_outstanding_claims_list.append(average_value_of_outstanding_claims)   
    average_value_of_outstanding_claims_list.append(average_value_of_outstanding_claims)
    print(i,iterate_months[i],'; value_of_outstanding_claims: ',value_of_outstanding_claims,'; outstanding_claims: ',outstanding_claims,'; value_of_reject_claims: ',value_of_reject_claims)
# Average Cost Per Claim


***

### Construct the report dataframe

In [ ]:
index_list=['Active Policies',
            'Live Policies',
            'New Inactive Policies',
            'Active Terminated Policies',
            'Gross Premium',
            'Gross Premium from Live Policies',
            'Gross Premium from New Inactive Policies',
            'Gross Premium from Active Terminated Policies',
            'New Policies',
            'GWP of New Policies',
            'Average Premium',
            'NTU',
            'Gross Premium for NTU',
            'Cancellations',
            'Gross Premium for Cancellations',
            'Number of Claims Received (Incl O/Docs Claims)',
            'Value of Claims Received',
            'Highest Claim Paid',
            'Number of Claims Paid',
            'Value of Claims Paid',
            'Average Cost Per Claim',
            'Repudiated Claims',
            'Value of Repudiated Claims',
            'Average Cost per repudiated claim',
            'Number of Oustanding Claims',
            'Value of Outstanding Claims',
           'Average Value of Outstanding Claims']


In [ ]:
data=[]
data.append(active_policies_list)
data.append(live_policies_list)
data.append(new_inactive_policies_list)
data.append(valid_terminations_list)
data.append(active_grosspremium_list)
data.append(live_policies_grosspremium_list)
data.append(new_policies_grosspremium_list)
data.append(valid_policies_grosspremium_list)

data.append(new_policies_list)
data.append(gwp_new_policies_list)
data.append(average_premium_list)
data.append(ntu_list)
data.append(gp_ntu_list)
data.append(cancelations_list)
data.append(gp_cancelations_list)
data.append(claims_received_list)
data.append(value_claims_received_list)
data.append(highest_claim_paid_list)
data.append(number_of_claims_paid_list)
data.append(value_of_claims_paid_list)

data.append(average_value_of_claims_paid_list)
data.append(reject_claims_list)
data.append(value_of_reject_claims_list)
data.append(average_value_of_reject_claims_list)
data.append(outstanding_claims_list)
data.append(value_of_outstanding_claims_list)
data.append(average_value_of_outstanding_claims_list)


In [ ]:
df = pd.DataFrame(data,index=index_list, columns=iterate_months[1:-1])

In [ ]:
for i in range(1,len(iterate_months)-1):
    print(iterate_months[i])

In [ ]:
iterate_months[1:-1]

In [ ]:
s=owls_members.split("_")[1].split(".")[0]
filename="cbr"+'_'+product+"_"+s+".csv"
df.to_csv(path_or_buf=report_path+filename, sep=',', index=True)

In [ ]:
presentation_path='\\\klo-jhb-fp01\\Xelus\\Data Science\\ChristoStrydom\\Presentations\\'
presentation_name='cbr_presentation_template.xlsx'
# file_name='cbr_presentation_template.xlsx'
cbr_presentation_template = openpyxl.load_workbook(presentation_path+presentation_name) 
summary_sheet = cbr_presentation_template["Sanlam Gap - Individual"]

In [ ]:
range_dict_mp={'Sanlam Gap - Individual':'SanlamGapIndividual_MP'}
# 'SanlamGapIndividual_MP'

In [ ]:
result_address = list(cbr_presentation_template.defined_names['SanlamGapIndividual_MP'].destinations)

In [ ]:
result_address

In [ ]:
df.loc[['Active Policies',
            'Live Policies',
            'New Inactive Policies',
            'Active Terminated Policies',
            'Gross Premium',
            'Gross Premium from Live Policies',
            'Gross Premium from New Inactive Policies',
            'Gross Premium from Active Terminated Policies',
            'New Policies',
            'GWP of New Policies',
            'Average Premium',
            'NTU',
            'Gross Premium for NTU',
            'Cancellations',
            'Gross Premium for Cancellations']]

In [ ]:
summary_sheet[result_address[0][1].replace('$', '')]

In [ ]:
df.reset_index()

In [ ]:
df.loc[['Active Policies',
            'Live Policies',
            'New Inactive Policies',
            'Active Terminated Policies',
            'Gross Premium',
            'Gross Premium from Live Policies',
            'Gross Premium from New Inactive Policies',
            'Gross Premium from Active Terminated Policies',
            'New Policies',
            'GWP of New Policies',
            'Average Premium',
            'NTU',
            'Gross Premium for NTU',
            'Cancellations',
            'Gross Premium for Cancellations']].reset_index()

In [ ]:
summary_sheet[result_address[0][1].replace('$', '')] = df.loc[['Active Policies',
            'Live Policies',
            'New Inactive Policies',
            'Active Terminated Policies',
            'Gross Premium',
            'Gross Premium from Live Policies',
            'Gross Premium from New Inactive Policies',
            'Gross Premium from Active Terminated Policies',
            'New Policies',
            'GWP of New Policies',
            'Average Premium',
            'NTU',
            'Gross Premium for NTU',
            'Cancellations',
            'Gross Premium for Cancellations']].reset_index()

In [ ]:
outstanding_claims_date_filter

In [ ]:
print(owls_received_claims_df[outstanding_claims_filter].shape[0])

In [ ]:
df.shape